## Qiskit Tutorials

Starting with Qiskit let's see how we can use the binders


### QLM to Qiskit conversion

We'll start with QLM to Qiskit conversion

In [2]:
from qat.interop.qiskit.converters import to_qiskit_circ
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

# Conversion
qiskit_circuit = to_qiskit_circ(qlm_circuit)


### Qiskit to QLM conversion

This conversion is going to be similar with every other language, the function `to_qlm_circ` is going to be the same for 
the other languages with two parameters: the circuit, and a boolean `sep_measure` if set to `True` it will return the qubits 
to measure separately for the user to measure them manually.
A tuple of 2 elements is then returned, the first element is the circuit with no measures, the second one is a list containing the qubits to be measured.
If set to `False` (which is the default value) only the QLM circuit will be returned (with measures included inside the circuit). 

In [4]:
from qat.interop.qiskit.converters import to_qlm_circ
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister

nbqubits = 2

qreg = QuantumRegister(nbqubits)
creg = ClassicalRegister(nbqubits)

qiskit_circuit = QuantumCircuit(qreg, creg)

qiskit_circuit.h(qreg[0])
qiskit_circuit.cx(qreg[0], qreg[1])
qiskit_circuit.measure(qreg, creg)

qlm_circuit_with_measure = to_qlm_circ(qiskit_circuit, sep_measure=False)
# This one is also equivalent because the default value of sep_measure is False
qlm_circuit_with_measure = to_qlm_circ(qiskit_circuit)

# This will return a list of qubits to measure manually as explained
qlm_circuit_without_measure, qubits_to_measure = to_qlm_circ(qiskit_circuit, sep_measure=True)

from qat.core.util import extract_syntax

for entry in qlm_circuit_without_measure.ops:
    # no need to print measures
    if entry.gate is None:
        continue
    print("Gate {} with params {} on qbits {}"
          .format(*extract_syntax(qlm_circuit_without_measure.gateDic[entry.gate],
                                qlm_circuit_without_measure.gateDic), entry.qbits))

Gate H with params [] on qbits [0]
Gate CNOT with params [] on qbits [0, 1]


### QiskitQPU

A wrapper implementing `QPUHandler` in order to run _synchronous_ QLM circuits in qiskit backends seamlessly and without any hassle

In [1]:
from qat.interop.qiskit.providers import QiskitQPU
from qiskit import Aer
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *

nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

qlm_job = qlm_circuit.to_job(nbshots=1024)

# Getting the qiskit backend we will use for our circuit execution
backend = Aer.get_backend('qasm_simulator')

qpu = QiskitQPU(backend)


result = qpu.submit(qlm_job)
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Use importlib.util.find_spec() instead.


State: |11>	 probability: 1.0


### AsyncQiskitQPU

We can also have the same thing but this time with an asynchronous qiskit qpu, this won't have the same interface as Qiskit,
but it will have the same interface as our QLM `Asyncjob`

In [2]:
from qat.interop.qiskit.providers import AsyncQiskitQPU
from qiskit import Aer
from qat.lang.AQASM import Program
from qat.lang.AQASM.gates import *
import time
nbqubits = 2

prog = Program()

qreg = prog.qalloc(nbqubits)
creg = prog.calloc(nbqubits)

prog.apply(H, qreg[0])
prog.apply(CNOT, qreg[0], qreg[1])

prog.measure(qreg, creg)
qlm_circuit = prog.to_circ()

qlm_job = qlm_circuit.to_job(nbshots=1024)

# Getting the qiskit backend we will use for our circuit execution
backend = Aer.get_backend('qasm_simulator')

qpu = AsyncQiskitQPU(backend)

async_job = qpu.submit_job(qlm_job)

print("ID: {}\t status : {}".format(async_job.job_id(), async_job.status()))
time.sleep(0.01)
print("ID: {}\t status : {}".format(async_job.job_id(), async_job.status()))
time.sleep(0.2)
print("ID: {}\t status : {}".format(async_job.job_id(), async_job.status()))
result = async_job.result()
for entry in result.raw_data:
    print("State: {}\t probability: {}".format(entry.state, entry.probability))

ID: 0daf18b7-bb24-45c3-a37a-cee675a44e33	 status : INITIALIZING
ID: 0daf18b7-bb24-45c3-a37a-cee675a44e33	 status : RUNNING
ID: 0daf18b7-bb24-45c3-a37a-cee675a44e33	 status : DONE
State: |0>	 probability: 1.0


### QLMBackend

We have implemented a qiskit style backend, that can be plugged seamlessly into qiskit's ecosystem. With this you can simulate qiskit circuits with any simulator you want that's available in the QLM (this of course includes other simulators like pyquil)

In [3]:
from qat.interop.qiskit.providers import QLMBackend
from qat.linalg import LinAlg
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute
from qat.comm.shared.ttypes import Batch
nbqubits = 2

qreg = QuantumRegister(nbqubits)
creg = ClassicalRegister(nbqubits)

qiskit_circuit = QuantumCircuit(qreg, creg)

qiskit_circuit.h(qreg[0])
qiskit_circuit.cx(qreg[0], qreg[1])
qiskit_circuit.measure(qreg, creg)


qpu = LinAlg()
backend = QLMBackend()
backend.set_qpu(qpu)

result = execute(qiskit_circuit, backend, shots=15).result()

from qat.interop.qiskit.providers import generate_qlm_result

qlm_result = generate_qlm_result(result)
    
for entry in qlm_result.raw_data:
     print("State: {}\t probability: {}".format(entry.state, entry.probability))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Use importlib.util.find_spec() instead.
  
/home/reda/.local/lib/python3.6/site-packages/qiskit/converters/qobj_to_circuits.py:34: DeprecationWarning: qiskit.converters.qobj_to_circuit() is deprecated and will be removed in Qiskit Terra 0.9. Please use qiskit.compiler.disassemble_circuits() to convert a qobj to list of circuits.
  'to list of circuits.', DeprecationWarning)


State: |0>	 probability: 0.5333333333333333
State: |11>	 probability: 0.4666666666666667


### QLMBackend using Pyquil Simulator

Let us put what we hypothesized earlier to the test, this time instead of using a native QLM simulator, we will use `PyquilQPU` the qpu wrapper around pyquil's qvm.


In [5]:
from qat.interop.qiskit.providers import QLMConnector
from qat.interop.pyquil.providers import PyquilQPU
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute
from pyquil.api import QVMConnection
nbqubits = 2

qreg = QuantumRegister(nbqubits)
creg = ClassicalRegister(nbqubits)

qiskit_circuit = QuantumCircuit(qreg, creg)

qiskit_circuit.h(qreg[0])
qiskit_circuit.cx(qreg[0], qreg[1])
qiskit_circuit.measure(qreg, creg)

# Connection to the qvm running as server
qvm = QVMConnection(endpoint="http://127.0.0.1:15011")


# Building the backend while using qiskit backend as a plugin
backend = QLMConnector | PyquilQPU(qvm)

# This result is in qiskit form since it was executed on qiskit
result = execute(qiskit_circuit, backend, shots=1024).result()

print(result.results)

[ExperimentResult(data=ExperimentResultData(counts=Obj(0x0=1)), header=Obj(clbit_labels=[['c4', 0], ['c4', 1]], creg_sizes=[['c4', 2]], memory_slots=2, n_qubits=2, name='circuit12', qreg_sizes=[['q4', 2]], qubit_labels=[['q4', 0], ['q4', 1]]), meas_level=2, shots=1, success=True)]


/home/reda/.local/lib/python3.6/site-packages/qiskit/converters/qobj_to_circuits.py:34: DeprecationWarning: qiskit.converters.qobj_to_circuit() is deprecated and will be removed in Qiskit Terra 0.9. Please use qiskit.compiler.disassemble_circuits() to convert a qobj to list of circuits.
  'to list of circuits.', DeprecationWarning)
